In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Rutas de entrada y salida
input_path = './data/l_vm_completa_normalizada_fe.parquet'
output_train = './data/df_train.parquet'
output_val = './data/df_val.parquet'

# Periodos para división
periodo_train_max = 201908
periodos_val = [201909, 201910]

In [2]:
output_train_limpio = './data/df_train_limpio.parquet'
output_val_limpio = './data/df_val_limpio.parquet'

df_train_limpio = pd.read_parquet(output_train_limpio, engine='fastparquet')
df_val_limpio = pd.read_parquet(output_val_limpio, engine='fastparquet')

In [4]:
from autogluon.tabular import TabularPredictor

# ============================
# Paso 1: Agregar sample_weight
# ============================

alpha = 1.0  # Podés ajustar este valor según la dispersión de tu variable

max_weight = 10.0
df_train_limpio['sample_weight'] = (1 + alpha * df_train_limpio['CLASE_DELTA_ZSCORE'].abs()).clip(upper=max_weight)
df_val_limpio['sample_weight'] = (1 + alpha * df_val_limpio['CLASE_DELTA_ZSCORE'].abs()).clip(upper=max_weight)


# ============================
# Paso 2: Entrenar el predictor
# ============================

predictor = TabularPredictor(
    label='CLASE_DELTA_ZSCORE',
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=4,
    path='AutogluonModels/nn_gpu_full_train_v2_weighted'
).fit(
    train_data=df_train_limpio,
    tuning_data=df_val_limpio,
    time_limit=7200,  # 2 horas
    use_bag_holdout=False,
    presets='medium',
    hyperparameters={
        'NN_TORCH': [{
            'num_epochs': 25,
            'learning_rate': 0.005,
            'dropout_prob': 0.1,
            'batch_size': 6144,
            'hidden_size': 1024,
            'ag_args': {'name_suffix': 'GPU_Full_Weighted'},
            'ag_args_fit': {
                'num_gpus': 1,
                'sample_weight': 'sample_weight'  # <--- Esto es lo correcto
            }
        }]
    }
)

Training data for TabularNeuralNetTorchModel has: 6730977 examples, 381 features (381 vector, 0 embedding)
Training on GPU
Neural network architecture:
EmbedNet(
  (main_block): Sequential(
    (0): Linear(in_features=381, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=1024, out_features=1024, bias=True)
    (10): ReLU()
    (11): Linear(in_features=1024, out_features=1, bias=True)
  )
)
Training tabular neural network for up to 25 epochs...
Epoch 1 (Update 1095).	Train loss: 5.7042, Val mean_absolute_error: -5.7107, Best Epoch: 1
Epoch 2 (Update 2190).	Train loss: 5.7089, Val mean_absolute_error: -5.7107, Best Epoch: 2
Epoch 3 (Update 3285).	Train loss: 5.7089, Val mean_absolute_error: 

In [5]:

# ===  Evaluación del modelo ===

lb = predictor.leaderboard(df_val_limpio, extra_metrics=['mean_absolute_error', 'median_absolute_error', 'r2'], silent=False)




Loading: /home/pablo/Documentos/labo3-2025v/AutogluonModels/nn_gpu_full_train_v2_weighted/models/NeuralNetTorchGPU_Full_Weighted/model.pkl
Loading: /home/pablo/Documentos/labo3-2025v/AutogluonModels/nn_gpu_full_train_v2_weighted/models/WeightedEnsemble_L2/model.pkl
Model scores:
{'NeuralNetTorchGPU_Full_Weighted': -0.11567343771457672, 'WeightedEnsemble_L2': -0.11567343771457672}


                             model  score_test  mean_absolute_error  median_absolute_error       r2  score_val          eval_metric  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  NeuralNetTorchGPU_Full_Weighted   -0.115673            -0.115673              -0.016925  0.89115  -0.115673  mean_absolute_error       12.732528      12.709661  1149.585235                12.732528               12.709661        1149.585235            1       True          1
1              WeightedEnsemble_L2   -0.115673            -0.115673              -0.016925  0.89115  -0.115673  mean_absolute_error       12.736080      12.711959  1149.594618                 0.003552                0.002298           0.009383            2       True          2


In [6]:
# Accedé al modelo dentro del predictor
model = predictor._trainer.load_model('NeuralNetTorchGPU_Full_Weighted')

# Inspeccionar los argumentos usados al entrenar
print(model.params)


Loading: /home/pablo/Documentos/labo3-2025v/AutogluonModels/nn_gpu_full_train_v2_weighted/models/NeuralNetTorchGPU_Full_Weighted/model.pkl


{'num_epochs': 25, 'epochs_wo_improve': None, 'activation': 'relu', 'embedding_size_factor': 1.0, 'embed_exponent': 0.56, 'max_embedding_dim': 100, 'y_range': None, 'y_range_extend': 0.05, 'dropout_prob': 0.1, 'optimizer': 'adam', 'learning_rate': 0.005, 'weight_decay': 1e-06, 'proc.embed_min_categories': 4, 'proc.impute_strategy': 'median', 'proc.max_category_levels': 100, 'proc.skew_threshold': 0.99, 'use_ngram_features': False, 'num_layers': 4, 'hidden_size': 1024, 'max_batch_size': 512, 'use_batchnorm': False, 'loss_function': 'auto', 'batch_size': 6144}


In [ ]:
feature_importance = predictor.feature_importance(df_val_limpio)


In [ ]:

# Imprimir las 20 características más importantes
print(feature_importance.tail(20))

✅ Paso 1: Calcular errores de predicción

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Predicciones y error absoluto
df_val_limpio['y_true'] = df_val_limpio['CLASE_DELTA_ZSCORE']
# Ignorar la columna 'CLASE_DELTA_ZSCORE' 
df_val = df_val_limpio.drop(columns=['CLASE_DELTA_ZSCORE'])
df_val_limpio['y_pred'] = predictor.predict(df_val_limpio)
df_val_limpio['error_abs'] = abs(df_val_limpio['y_true'] - df_val_limpio['y_pred'])
df_val_limpio['error_signed'] = df_val_limpio['y_pred'] - df_val_limpio['y_true']


📊 Paso 2: Histogramas de error absoluto y error signado

In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(df_val_limpio['error_abs'], bins=50, kde=True)
plt.title('Distribución del Error Absoluto')
plt.xlabel('|y_pred - y_true|')
plt.ylabel('Frecuencia')
plt.show()

plt.figure(figsize=(10, 4))
sns.histplot(df_val_limpio['error_signed'], bins=50, kde=True)
plt.title('Distribución del Error Signado')
plt.xlabel('y_pred - y_true')
plt.ylabel('Frecuencia')
plt.axvline(0, color='red', linestyle='--')
plt.show()


📈 Paso 3: Error vs. Valor real (dispersión)


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_val_limpio['y_true'], y=df_val_limpio['error_signed'], alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.title('Error Signado vs. Valor Real')
plt.xlabel('Valor real')
plt.ylabel('Error (pred - real)')
plt.show()


🧩 Paso 4: Promedio de error por grupo 

In [ ]:
top_errores_producto = df_val.groupby('PRODUCT_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 PRODUCT_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()


In [ ]:
top_errores_producto = df_val.groupby('CUSTOMER_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 CUSTOMER_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()

In [ ]:
df_full = pd.concat([df_train_limpio, df_val_limpio], ignore_index=True)
del df_train_limpio, df_val_limpio
gc.collect()
predictor_full = predictor_nn_lightfast.refit_full(train_data=df_full)
predictor_full.save("AutogluonModels/nn_lightfast_full")

In [ ]:
""" # Combinar entrenamiento + validación
df_full = pd.concat([df_train, df_val], axis=0)
del df_train, df_val
gc.collect() """

In [ ]:
# # Reentrenar el mejor modelo con TODOS los datos disponibles
# predictor_full = predictor.refit_full(train_data=df_full)

In [ ]:
# Verificar los modelos disponibles (el mejor ahora tiene el sufijo '_FULL')
print("Modelos disponibles luego del refit completo:")
print(predictor.leaderboard(silent=True)['model'].tolist())
# Eliminar modelos intermedios para liberar espacio
predictor.delete_models(models_to_keep='best', dry_run=False)

# Confirmar que solo queda el modelo reentrenado
print("\nModelos restantes después de eliminar los intermedios:")
print(predictor.leaderboard(silent=True)['model'].tolist())

# (Opcional) Guardar el predictor final si querés usarlo luego sin volver a cargar todo
predictor.save('./data/modelo_final_autogluon')

# ---  Liberar memoria ---
del df_full
gc.collect()


In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones
df_pred_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')
# Dejo solo los datos del periodo 201910 y que A_PREDECIR sea True
# Filtrar solo los datos del periodo 201910 y donde A_PREDECIR sea True
df_pred_full = df_pred_full[
    (df_pred_full['PERIODO'] == 201910) & (df_pred_full['A_PREDECIR'] == True)
].drop(columns=['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE'])

In [ ]:
# Realizar las predicciones usando el predictor original
predictions = predictor.predict(df_pred_full)
# Agregar las predicciones al DataFrame original
df_pred_full['CLASE_DELTA_ZSCORE'] = predictions

In [ ]:
# Imprimir la lista de columas del DataFrame con las predicciones
print("Columnas del DataFrame con las predicciones:")
print(df_pred_full.columns.tolist())

In [ ]:
# Dernormalizar la columna CLASE_DELTA_ZSCORE
df_pred_full['CLASE_DELTA'] = df_pred_full['CLASE_DELTA_ZSCORE'] * df_pred_full['CLASE_DELTA_STD'] + df_pred_full['CLASE_DELTA_MEAN']
df_pred_full['TN'] = df_pred_full['TN_ZSCORE'] * df_pred_full['TN_STD'] + df_pred_full['TN_MEAN']
# Agregar la columna TN_PREDICT que sea la suma de TN y CLASE_DELTA y si es menor que cero, poner cero
df_pred_full['TN_PREDICT'] = df_pred_full['TN'] + df_pred_full['CLASE_DELTA']
df_pred_full['TN_PREDICT'] = df_pred_full['TN_PREDICT'].clip(lower=0)

In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de TN_PREDICT
df_final = df_pred_full.groupby('PRODUCT_ID').agg({'TN_PREDICT': 'sum'}).reset_index()
df_final = df_final.rename(columns={'PRODUCT_ID': 'product_id', 'TN_PREDICT': 'tn'})
# Guardar el DataFrame df_final en un archivo CSV
df_final.to_csv('./modelos/autoglun_normalizando_clase_delta.csv', index=False)
df_final.shape

# Para instalar AutoGluon con soporte para `autogluon.core.space` en conda, ejecuta:
# 
# conda install -c conda-forge autogluon
# 
# O si prefieres usar pip dentro de tu entorno conda:
# 
# pip install autogluon
# 
# Luego podrás usar:
# from autogluon.core import space as ag